In [ ]:
!pip install requests
!pip install opencv-python
!pip install mediapipe
!pip install pygame

In [77]:
import cv2
import mediapipe as mp
import time
import requests
import math
import pygame
import numpy as np
import io
from pathlib import Path
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:

# ===== INIZIALIZZAZIONE PYGAME =====
pygame.init()
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)
pygame.mixer.set_num_channels(2)  # più canali per effetti multipli
channel_volume = pygame.mixer.Channel(0)
channel_gesture = pygame.mixer.Channel(1)


# ===== VARIABILI GLOBALI =====
# Stato del player
is_paused = False
playlist = []
current_song_index = 0
volume = 0.5

# Percorsi dei file audio per i feedback sonori
volume_sounds = {}
for i in range(11): 
    volume_sounds[i] = pygame.mixer.Sound(f"feedbacksounds/vol_{i}.wav")
path_completed_sound = r"feedbacksounds\stop.wav"

# Initialize pygame mixer
# Load the sound file
GESTURE_COMPLETED_SOUND = pygame.mixer.Sound(path_completed_sound)
# Set volume (0.0 to 1.0)
GESTURE_COMPLETED_SOUND.set_volume(1)


# Variabili per il rilevamento gesti
last_trigger_time = 0
detection_start_time = 0
is_detecting = False
current_gesture = None
last_landmarks_position = None
consecutive_thumbs_up_count = 0
consecutive_thumbs_down_count = 0
last_gesture = None

# Zone di attivazione (distanze per rilevare se la mano è nella zona corretta)
min_dist_medio = 0.15
min_dist_pollice = 0.20
max_dist_medio = 0.35
max_dist_pollice = 0.45


In [79]:

# ===== FUNZIONI MUSICALI =====
def search_songs(song_titles):
    """Cerca canzoni su Deezer e costruisce la playlist"""
    global playlist
    playlist = []
    
    for title in song_titles:
        try:
            url = f"https://api.deezer.com/search?q={title}"
            response = requests.get(url, timeout=5)
            data = response.json()
            
            if data.get('data'):
                track = data['data'][0]
                song_info = {
                    "title": track['title'],
                    "artist": track['artist']['name'],
                    "preview_url": track['preview']
                }
                playlist.append(song_info)
                print(f"🎶 Aggiunto: {song_info['title']} - {song_info['artist']}")
            else:
                print(f"⚠️ Nessun risultato per: {title}")
        except Exception as e:
            print(f"❌ Errore nella ricerca di '{title}': {e}")

def load_song(index):
    """Carica e riproduce una canzone dall'indice specificato"""
    global volume
    
    if not (0 <= index < len(playlist)):
        print("❌ Indice fuori range.")
        return False
        
    try:
        url = playlist[index]['preview_url']
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            audio = io.BytesIO(response.content)
            pygame.mixer.music.load(audio)
            pygame.mixer.music.set_volume(volume)
            pygame.mixer.music.play()
            
            song = playlist[index]
            print(f"▶️ Riproduzione: {song['title']} - {song['artist']}")
            return True
        else:
            print("❌ Errore nel caricamento della preview.")
            return False
    except Exception as e:
        print(f"❌ Errore nel caricamento: {e}")
        return False

def play_pause():
    """Alterna tra play e pausa"""
    global is_paused
    
    if is_paused:
        pygame.mixer.music.unpause()
        is_paused = False
        print("▶️ Ripresa")
    else:
        pygame.mixer.music.pause()
        is_paused = True
        print("⏸️ Pausa")

def next_song():
    """Passa alla canzone successiva"""
    global current_song_index, is_paused
    
    if len(playlist) == 0:
        print("❌ Playlist vuota")
        return
        
    current_song_index = (current_song_index + 1) % len(playlist)
    load_song(current_song_index)
    is_paused = False

def previous_song():
    """Passa alla canzone precedente"""
    global current_song_index, is_paused
    
    if len(playlist) == 0:
        print("❌ Playlist vuota")
        return
        
    current_song_index = (current_song_index - 1) % len(playlist)
    load_song(current_song_index)
    is_paused = False

def volume_up():
    """Aumenta il volume"""
    global volume
    volume = min(1.0, round(volume + 0.1, 1))
    pygame.mixer.music.set_volume(volume)
    print(f"🔊 Volume: {int(volume * 100)}%")

def volume_down():
    """Diminuisce il volume"""
    global volume
    volume = max(0.0, round(volume - 0.1, 1))
    pygame.mixer.music.set_volume(volume)
    print(f"🔉 Volume: {int(volume * 100)}%")


In [80]:

# # ===== FUNZIONI PER IL RILEVAMENTO GESTI =====
def is_active_zone(hand_landmarks):
    """Verifica se la mano è nella zona attiva per il rilevamento"""
    # Calcola distanze
    wrist_middle = math.dist(
        [hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y],
        [hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y]
    )
    wrist_thumb = math.dist(
        [hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y],
        [hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y]
    )
    knuckles_dist = math.dist(
        [hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y],
        [hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y]
    )
    
    return ((min_dist_medio < wrist_middle < max_dist_medio) or 
            (min_dist_pollice < wrist_thumb < max_dist_pollice)) and \
           (knuckles_dist > 0.08)



def estimate_hand_side(landmarks, handedness):
    """Stima se stiamo vedendo il palmo o il dorso della mano"""
    WRIST = 0
    INDEX_MCP = 5
    PINKY_MCP = 17
    
    wrist = np.array([landmarks[WRIST].x, landmarks[WRIST].y, landmarks[WRIST].z])
    index_mcp = np.array([landmarks[INDEX_MCP].x, landmarks[INDEX_MCP].y, landmarks[INDEX_MCP].z])
    pinky_mcp = np.array([landmarks[PINKY_MCP].x, landmarks[PINKY_MCP].y, landmarks[PINKY_MCP].z])
    
    vec1 = index_mcp - wrist
    vec2 = pinky_mcp - wrist
    normal = np.cross(vec1, vec2)
    
    if handedness == 'Left':
        normal = -normal
    
    return "PALM" if normal[2] > 0 else "BACK"



In [81]:
# Parametri gesture recognition
cooldown = 0.7  # Delay tra un gesto e l'altro (in secondi)
detection_delay = 0.7 # Delay tra gesto ed azione (in secondi)
movement_threshold = 0.05  # Soglia di movimento accettabile
cam = 0  # 0 per webcam principale, 1 per secondaria

In [ ]:
def handle_gesture(name, action_fn, hand_landmarks, frame):
    """Gestisce il rilevamento e l'esecuzione dei gesti"""
    global last_trigger_time, detection_start_time, is_detecting, current_gesture
    global consecutive_thumbs_up_count, consecutive_thumbs_down_count, last_gesture
    global is_paused, last_landmarks_position, movement_threshold, volume
    
    # Controllo stabilità del movimento
    if is_detecting and current_gesture == name:
        current_center_x = hand_landmarks.landmark[0].x  # polso
        current_center_y = hand_landmarks.landmark[0].y
        
        if last_landmarks_position is not None:
            distance = math.dist([current_center_x, current_center_y], last_landmarks_position)
            
            if distance > movement_threshold:
                print(f"❌ Movimento eccessivo per {name}. Reset.")
                is_detecting = False
                current_gesture = None
                detection_start_time = 0
                last_landmarks_position = None
                return
    
    # Inizio nuovo rilevamento
    if not is_detecting or current_gesture != name:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        last_landmarks_position = [hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y]
    
    elapsed_time = time.time() - detection_start_time
    
    # Visualizzazione del gesto e progress
    x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), \
           int(hand_landmarks.landmark[0].y * frame.shape[0])
    
    # Mostra nome del gesto
    #d_t= displayed text
    if name == "Open_Palm":
        d_t="PLAY/PAUSE"
    elif name == "Thumb_Up":
        d_t="VOLUME UP"
    elif name == "Thumb_Down":
        d_t="VOLUME DOWN"
    elif name == "Victory":
        d_t="NEXT SONG"
    elif name == "Closed_Fist":
        d_t="PREVIOUS SONG"

    cv2.putText(frame, d_t, (350, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Cerchio di caricamento
    radius = 30
    center = (x, y - 20)
    cv2.circle(frame, center, radius, (100, 100, 100), 3)
    
    # Progresso del caricamento
    angle = min(elapsed_time / detection_delay * 360, 360)
    start_angle = -90
    
    for i in range(int(start_angle), int(start_angle + angle)):
        rad = math.radians(i)
        x1 = int(center[0] + radius * math.cos(rad))
        y1 = int(center[1] + radius * math.sin(rad))
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), 3)
    
    # Esecuzione del gesto quando completato
    if elapsed_time >= detection_delay:
        action_fn()
        print(f"✅ {name.upper()} ESEGUITO!")

        print("consecutive_thumbs_up_count:", consecutive_thumbs_up_count)
        # Aggiorna contatori per gesti consecutivi
        if name == "Thumb_Up":
            if last_gesture == "Thumb_Up":
                consecutive_thumbs_up_count += 1
                channel_volume.play(volume_sounds[int(volume*10)])  
            else:
                consecutive_thumbs_up_count = 1
                consecutive_thumbs_down_count = 0
                channel_volume.play(volume_sounds[int(volume*10)])  
            
        elif name == "Thumb_Down":
            if last_gesture == "Thumb_Down":
                consecutive_thumbs_down_count += 1
                channel_volume.play(volume_sounds[int(volume*10)])  
            else:
                consecutive_thumbs_down_count = 1
                consecutive_thumbs_up_count = 0
                channel_volume.play(volume_sounds[int(volume*10)]) 

        else:
            # Per tutti gli altri gesti, reset contatori e riproduci suono standard
            consecutive_thumbs_up_count = 0
            consecutive_thumbs_down_count = 0
            channel_gesture.play(GESTURE_COMPLETED_SOUND)

        
        last_gesture = name
        last_trigger_time = time.time()
        is_detecting = False
        current_gesture = None
        last_landmarks_position = None


In [83]:

# ===== INIZIALIZZAZIONE PLAYLIST =====
# Modifica questa lista con le tue canzoni preferite
song_titles = [
    "Bohemian Rhapsody Queen",
    "Around the World Daft Punk", 
    "Blinding Lights The Weeknd",
    "Imagine John Lennon"
]

print("🔍 Ricerca canzoni in corso...")
search_songs(song_titles)

if playlist:
    print(f"✅ Playlist creata con {len(playlist)} canzoni")
    load_song(current_song_index)
    is_paused = False
else:
    print("❌ Nessuna canzone trovata. Controlla la connessione internet.")

# ===== INIZIALIZZAZIONE MEDIAPIPE =====
print("🤖 Inizializzazione MediaPipe...")

# Carica il modello per il riconoscimento gesti
model_path = Path("gesture_recognizer.task").resolve()

if not model_path.exists():
    print("❌ File 'gesture_recognizer.task' non trovato!")
    print("Scarica il modello da: https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer")
else:
    try:
        base_options = python.BaseOptions(model_asset_buffer=open(model_path, "rb").read())
        options = vision.GestureRecognizerOptions(base_options=base_options)
        recognizer = vision.GestureRecognizer.create_from_options(options)
        
        # Inizializzazione MediaPipe Hands
        mp_hands = mp.solutions.hands
        mp_draw = mp.solutions.drawing_utils
        hands = mp_hands.Hands(
            max_num_hands=1,
            model_complexity=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )
        
        print("✅ MediaPipe inizializzato correttamente")
        
    except Exception as e:
        print(f"❌ Errore nell'inizializzazione MediaPipe: {e}")




🔍 Ricerca canzoni in corso...
🎶 Aggiunto: Bohemian Rhapsody (Remastered 2011) - Queen
🎶 Aggiunto: Around the World - Daft Punk
🎶 Aggiunto: Blinding Lights - The Weeknd
🎶 Aggiunto: Imagine (Remastered 2010) - John Lennon
✅ Playlist creata con 4 canzoni
▶️ Riproduzione: Bohemian Rhapsody (Remastered 2011) - Queen
🤖 Inizializzazione MediaPipe...
✅ MediaPipe inizializzato correttamente


In [84]:

# ===== LOOP PRINCIPALE =====
print("🎥 Avvio webcam...")
print("📋 Gesti disponibili:")
print("   👋 Mano aperta (palmo) = Play/Pausa")
print("   👍 Thumbs up = Volume su")
print("   👎 Thumbs down = Volume giù")
print("   ✌️ Victory = Canzone successiva")
print("   ✊ Pugno chiuso = Canzone precedente")
print("   ❌ Premi 'q' per uscire")

cap = cv2.VideoCapture(cam)
if not cap.isOpened():
    print("❌ Impossibile aprire la webcam")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("❌ Impossibile leggere dal dispositivo video")
            break
        
        # Reset rilevamento se troppo tempo è passato
        if is_detecting and (time.time() - detection_start_time > detection_delay * 1.5):
            is_detecting = False
            current_gesture = None
        
        # Elaborazione frame
        frame = cv2.flip(frame, 1)  # Specchia l'immagine
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        results = hands.process(rgb_frame)
        
        # Elaborazione risultati
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                
                # Determina il colore in base alla zona attiva
                if is_active_zone(hand_landmarks):
                    landmark_color = (0, 255, 0)  # Verde
                    connection_color = (100, 100, 100)
                else:
                    landmark_color = (0, 0, 255)  # Rosso
                    connection_color = (0, 0, 255)
                
                # Disegna la mano
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_draw.DrawingSpec(color=landmark_color, thickness=2, circle_radius=2),
                    mp_draw.DrawingSpec(color=connection_color, thickness=2, circle_radius=2)
                )
                
                # Riconoscimento gesti
                try:
                    result = recognizer.recognize(mp_image)
                    
                    if result.gestures:
                        gesture = result.gestures[0][0]
                        gesture_name = gesture.category_name
                        score = gesture.score
                        
                        # Verifica cooldown e zona attiva
                        skip_cooldown = False
                        if ((last_gesture == "Thumb_Up" and consecutive_thumbs_up_count >= 2) or
                            (last_gesture == "Thumb_Down" and consecutive_thumbs_down_count >= 2)):
                            skip_cooldown = True
                        
                        if (is_active_zone(hand_landmarks) and 
                            ((time.time() - last_trigger_time) > cooldown or skip_cooldown)):
                            
                            # Gestione dei diversi gesti
                            if (gesture_name == "Open_Palm" and 
                                estimate_hand_side(hand_landmarks.landmark, 
                                                 hand_handedness.classification[0].label) == "PALM"):
                                handle_gesture("Open_Palm", play_pause, hand_landmarks, frame)
                            
                            elif gesture_name == "Thumb_Up":
                                handle_gesture("Thumb_Up", volume_up, hand_landmarks, frame)
                            
                            elif gesture_name == "Thumb_Down":
                                handle_gesture("Thumb_Down", volume_down, hand_landmarks, frame)
                            
                            elif gesture_name == "Victory":
                                handle_gesture("Victory", next_song, hand_landmarks, frame)
                            
                            elif gesture_name == "Closed_Fist":
                                handle_gesture("Closed_Fist", previous_song, hand_landmarks, frame)
                        
                        # Mostra info gesto riconosciuto
                        cv2.putText(frame, f"{gesture_name} ({score:.2f})", (10, 50),
                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                except Exception as e:
                    print(f"❌ Errore nel riconoscimento gesti: {e}")
        
        # Mostra info corrente
        if playlist and current_song_index < len(playlist):
            current_song = playlist[current_song_index]
            info_text = f"-> {current_song['title']} - {current_song['artist']} | Vol: {int(volume*100)}%"
            cv2.putText(frame, info_text, (10, frame.shape[0] - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow("Gesture Music Controller", frame)
        
        # Uscita con 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Cleanup
    pygame.mixer.music.stop()
    cap.release()
    cv2.destroyAllWindows()
    print("👋 Arrivederci!")

🎥 Avvio webcam...
📋 Gesti disponibili:
   👋 Mano aperta (palmo) = Play/Pausa
   👍 Thumbs up = Volume su
   👎 Thumbs down = Volume giù
   ✌️ Victory = Canzone successiva
   ✊ Pugno chiuso = Canzone precedente
   ❌ Premi 'q' per uscire
❌ Movimento eccessivo per Thumb_Up. Reset.
❌ Movimento eccessivo per Thumb_Up. Reset.
❌ Movimento eccessivo per Thumb_Up. Reset.
🔊 Volume: 60%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 0
🔊 Volume: 70%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 1
🔊 Volume: 80%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 2
🔊 Volume: 90%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 3
🔊 Volume: 100%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 4
🔊 Volume: 100%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 5
🔊 Volume: 100%
✅ THUMB_UP ESEGUITO!
consecutive_thumbs_up_count: 6
▶️ Riproduzione: Around the World - Daft Punk
✅ VICTORY ESEGUITO!
consecutive_thumbs_up_count: 7
▶️ Riproduzione: Blinding Lights - The Weeknd
✅ VICTORY ESEGUITO!
consecuti